In [10]:
import geopandas as gpd
from shapely.geometry import Polygon
import folium
import s2sphere
import pandas as pd
import numpy as np
from my_functions import s2_latlng_to_polygon

# set some variables
data_path = 'data/daily_by_station.gz.parquet'
crs = {'init': 'epsg:4326'}
map_center = [53.4, -7.9]

## Read Data

In [33]:
# get data
df = pd.read_parquet(data_path)
print(df.shape)
df.head()

(2700980, 110)


dly_cbl_mean  dly_ddhm_mean  dly_dos_mean  \
date       station_number                                              
1941-01-01 108                      NaN            NaN           NaN   
           201                      NaN            NaN           NaN   
           405                      NaN            NaN           NaN   
           417                      NaN            NaN           NaN   
           418                      NaN            NaN           NaN   

                           dly_evap_mean  dly_g_rad_mean  dly_glorad_mean  \
date       station_number                                                   
1941-01-01 108                       NaN             NaN              NaN   
           201                       NaN             NaN              NaN   
           405                       NaN             NaN              NaN   
           417                       NaN             NaN              NaN   
           418                       NaN             NaN              NaN   

                           dly_gmin_mean  dly_hg_mean  dly_hm_mean  \
date       station_number                                            
1941-01-01 108                       NaN          NaN          NaN   
           201                       NaN          NaN          NaN   
           405                       NaN          NaN          NaN   
           417                       NaN          NaN          NaN   
           418                       NaN          NaN          NaN   

                           dly_igmin_mean  ...  hly_wetb_amax  hly_wetb_amin  \
date       station_number                  ...                                 
1941-01-01 108                        NaN  ...            NaN            NaN   
           201                        NaN  ...            NaN            NaN   
           405                        NaN  ...            NaN            NaN   
           417                        NaN  ...            NaN            NaN   
           418                        NaN  ...            NaN            NaN   

                           hly_wetb_mean  hly_wetb_std  hly_ww_amax  \
date       station_number                                             
1941-01-01 108                       NaN           NaN          NaN   
           201                       NaN           NaN          NaN   
           405                       NaN           NaN          NaN   
           417                       NaN           NaN          NaN   
           418                       NaN           NaN          NaN   

                           hly_ww_amin  hly_ww_mean  hly_ww_std  latitude  \
date       station_number                                                   
1941-01-01 108                     NaN          NaN         NaN    52.311   
           201                     NaN          NaN         NaN    51.735   
           405                     NaN          NaN         NaN    51.978   
           417                     NaN          NaN         NaN    52.842   
           418                     NaN          NaN         NaN    52.780   

                           longitude  
date       station_number             
1941-01-01 108                -6.766  
           201                -9.546  
           405                -9.622  
           417                -9.238  
           418                -8.903  

[5 rows x 110 columns]

In [43]:
df[df.index.get_level_values('date') >= pd.to_datetime('2000-01-01 00:00')]

TypeError: Cannot compare type 'Timestamp' with type 'date'

## Plot Stations

In [35]:
# get list of lat long tuples for each station
latlng_list = list(set(zip(df.index.get_level_values('station_number'),zip(df['latitude'],df['longitude']))))

# plot of map
m = folium.Map(map_center, zoom_start=6.5, tiles='cartodbpositron')
for station_number, latlng in latlng_list:
    polygon_geom = Polygon(s2_latlng_to_polygon(latlng,parent_level=30))
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
    polygon['color'] = np.random.choice(['black'])
    polygon['opacity'] = np.random.choice([0.9])
    polygon['weight'] = np.random.choice([1])
    folium.GeoJson(polygon,style_function=lambda feature: {
        'fillColor': feature['properties']['color'],
        'color' : feature['properties']['color'],
        #'weight' : feature['properties']['weight'],
        #'fillOpacity' : feature['properties']['opacity'],
        }).add_child(folium.Popup(f'station #{station_number}')).add_to(m)
m